In [7]:
import tensorflow as tf
import os
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
dataDir = "Downloads/catDog" # catDog file contains two files for cat and dog images respectively
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
dataDir = "Downloads/catDog"
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

# deleteCnt = 0
for imageClass in os.listdir(dataDir):
    test = str(imageClass)
    if test == '.DS_Store': # some invisible file I can't delete manually
        continue
        
    for image in os.listdir(os.path.join(dataDir, imageClass)):
        imagePath = os.path.join(dataDir, imageClass, image)
        
        try:
          # removes images that are not compatible
          img = cv2.imread(imagePath)
          tip = imghdr.what(imagePath)
          if tip not in image_exts:
              print("Image not in list of extensions".format(imagePath))
              # os.remove(imagePath) this function does not trash images, it deletes
              # deleteCnt += 1
           
          # error will be caught if image is corrupted
          img_bytes = tf.io.read_file(path)
          decoded_img = tf.io.decode_image(img_bytes)
        
        except Exception as e:
          #os.remove(imagePath)
          deleteCnt += 1
            
# print(deleteCnt)
    

In [ ]:
import glob


for imageClass in os.listdir(dataDir): # removes imgs that aren't compatible
  for image_path in os.listdir(os.path.join(dataDir, imageClass)):
    path = os.path.join(dataDir, imageClass, image_path)
    try:
      img_bytes = tf.io.read_file(path)
      decoded_img = tf.io.decode_image(img_bytes)
    except tf.errors.InvalidArgumentError as e:
      print("Found!")
      os.remove(path)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# import tensorflow as tf

# stores images in batches, each batch has 32 images, images are stored as a 2D array of integers
ds = tf.keras.utils.image_dataset_from_directory(dataDir) # shape of each batch: [image][label ([0,1])]
#length of ds = number of images/32 

ds = ds.map(lambda x,y: (x/255, y)) # divides all pixel values ([0, 255] inclusive) by 255

# training data will be 80%, validation data will be 20%, in number of batches
trainSize = int(len(ds) * .8)+1
valSize = int(len(ds) * .2)

# we can use an iterator to traverse through batches
trainDS = ds.take(trainSize)
valDS = ds.skip(trainSize).take(valSize)

In [ ]:
ds = tf.keras.utils.image_dataset_from_directory(dataDir)

In [ ]:
ds = ds.map(lambda x,y: (x/255, y))

In [ ]:
trainSize = int(len(ds) * .8)+1
valSize = int(len(ds) * .2)

# 80% training, 20% validation

In [ ]:
trainDS = ds.take(trainSize)
valDS = ds.skip(trainSize).take(valSize)

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, RandomFlip, RandomRotation, RandomZoom, BatchNormalization
from tensorflow import keras
from keras import layers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [4]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
    input_shape=(256,256,3)),
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.5),
  ]
)

In [ ]:
# to reset training
del model
from keras import backend as K
K.clear_session()

In [2]:
model = Sequential()

2022-12-25 22:49:17.253274: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# augmentation will not be called for predictions
data_augmentation
model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape=(256, 256, 3)))
model.add(layers.Conv2D(32, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))

# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 32)      9248      
                                                                 
 batch_normalization (BatchN  (None, 252, 252, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      1

In [ ]:
learningRateReduction = ReduceLROnPlateau(monitor = 'val_accuracy', 
                                          patience = 1, 
                                          verbose = 1, 
                                          factor = .5, 
                                          min_lr = .00001)

earlystop = EarlyStopping(monitor='val_accuracy', patience=3)

callbacks = [learningRateReduction]

model.compile(optimizer = "adam", loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
hist = model.fit(trainDS, epochs = 50, validation_data = valDS, callbacks = callbacks)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='red', label='loss')
plt.plot(hist.history['val_loss'], color='blue', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper right")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='red', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='blue', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="lower right")
plt.show()

In [65]:
testImg = cv2.imread("Downloads/dogs/dog3.jpg")

In [66]:
resize = tf.image.resize(testImg, (256,256))

In [67]:
ans = model.predict(np.expand_dims(resize/255, 0))

1/1 [==============================] - 0s 64ms/step


In [ ]:
if ans <= .5:
    print("Its a cat!")
else:
    print("Not a cat!")
    
print(ans)

# cat1: 0.0625807
# cat2: 0.00000000000014656887
# cat3: 0.996319
# dog1: 1
# dog2: 0.23557787

In [ ]:
model.save("model/catOrDog.h5")

In [17]:
model.load_weights("model/catOrDog.h5")